## Explain LSTM->XGB on TBI signal data

Explain XGBoost model trained on LSTM embedded data

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import numpy as np
from tbi_downstream_prediction import split_data

import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential, load_model, Model
from matplotlib import cm, pyplot as plt
from sklearn import metrics
from os.path import expanduser as eu
from os.path import isfile, join
import numpy as np
import random, time

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
DPATH = PATH+"tbi/processed_data/hypoxemia/"
data_type = "raw[top11]"

feat_lst = ["ECGRATE", "ETCO2", "ETSEV", "ETSEVO", "FIO2", "NIBPD", "NIBPM", 
            "NIBPS","PEAK", "PEEP", "PIP", "RESPRATE", "SAO2", "TEMP1", "TV"]

# Exclude these features
weird_feat_lst = ["ETSEV", "PIP", "PEEP", "TV"]
feat_inds = np.array([feat_lst.index(feat) for feat in feat_lst if feat not in weird_feat_lst])
feat_lst2 = [feat for feat in feat_lst if feat not in weird_feat_lst]

y_tbi = np.load(DPATH+"tbiy.npy")
X_tbi = np.load(DPATH+"X_tbi_imp_standard.npy")

X_tbi2 = X_tbi[:,feat_inds,:]
(X_test, y_test, X_valid, y_valid, X_train, y_train) = split_data(DPATH,X_tbi2,y_tbi,flatten=False)

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
RESULTPATH = PATH+"results/"
label_type = "desat_bool92_5_nodesat"
lstm_type = "biglstmdropoutv3_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, lstm_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)
GPUNUM = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))

### Load model and generate independent explanations

In [ ]:
import shap
import xgboost as xgb

from tbi_downstream_prediction import *
PATH = "/homes/gws/hughchen/phase/tbi_subset/"
DPATH = PATH+"tbi/processed_data/hypoxemia/"
RESULTPATH = PATH+"results/"

# Load data
X_train = np.load(DPATH+"X_train_embed_lstmbigdropoutv3_50n_arr.npy",mmap_mode="r")
X_valid = np.load(DPATH+"X_valid_embed_lstmbigdropoutv3_50n_arr.npy",mmap_mode="r")
X_test  = np.load(DPATH+"X_test_embed_lstmbigdropoutv3_50n_arr.npy",mmap_mode="r")

# Set important variables
label_type = "desat_bool92_5_nodesat"; eta = 0.02
hosp_data = "tbi"; data_type = "lstm_big_50n[top11]"
mod_type = "xgb_{}_eta{}".format(label_type,eta)

# Set up result directory
RESDIR = '{}results/{}/'.format(PATH, mod_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

# Set parameters to train model
param = {'max_depth':6, 'eta':eta, 'subsample':0.5, 'gamma':1.0, 
         'min_child_weight':10, 'base_score':y_train.mean(), 
         'objective':'binary:logistic', 'eval_metric':["logloss"]}

save_path = RESDIR+"hosp{}_data/{}/".format(hosp_data,data_type)
bst = xgb.Booster()
bst.load_model(save_path+'mod_eta{}.model'.format(param['eta']))
X_tbi2 = X_tbi[:,feat_inds,:]
(_, _, _, _, X_train_raw, _) = split_data(DPATH,X_tbi2,y_tbi,flatten=False)

# Generate explanations
import numpy as np
np.random.seed(10)
background_inds = np.random.choice(np.arange(0,X_train.shape[0]),1000)
X_train = np.load(DPATH+"X_train_embed_lstmbigdropoutv2_arr.npy")
X_train_background = X_train[background_inds]
ind_explainer = shap.TreeExplainer(bst,data=X_train_background,feature_dependence="independent")
X_train_ind_explanations = ind_explainer.shap_values(X_train)
np.save(save_path+"X_train_explanations_ind",X_train_ind_explanations)

#### Plot the aggregated explanations

In [ ]:
X_train_ind_explanations_agg = np.transpose(np.vstack([X_train_ind_explanations[:,i*50:(i+1)*50].sum(1) for i in range(len(feat_lst2))]))
shap.summary_plot(X_train_ind_explanations_agg,X_train_raw.sum(1),feature_names=feat_lst2)